In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()
output_null = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./kit_part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error, output_null
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://www.rockauto.com/en/parts/' + input_.loc[a, 'Kit_Manufacturer'].replace('+', '%2B').replace(',', '%2C') + ',' + input_.loc[a, 'Kit_Part_Number'] + ',Lennon,' + input_.loc[a, 'Kit_Type_Code']

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =

                no_parts_found = html.xpath('//span[@class="navlabellink nvoffset nnormal"]/text()')

                if len(no_parts_found) != 0 and no_parts_found[0].strip() == 'No Parts Found':
                    crawler_status = 'null'
                    break
                    
                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_json = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_supplemental")]/@value')[0]) for part in list_part]

                # = = = = = = = = = = = = = = =

                index_part = -1
                for i in range(len(list_json)):
                    if list_json[i]['catalogname'].strip() == input_.loc[a, 'Kit_Manufacturer'] and list_json[i]['partnumber'].strip() == input_.loc[a, 'Kit_Part_Number']:
                        if index_part != -1:
                            continue
                        index_part = i

                # = = = = = = = = = = = = = = =

                category = html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                type_ = html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_note_1 = list_part[index_part].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                note_1 = '' if len(list_note_1) == 0 else ';'.join([note_1.strip() for note_1 in list_note_1])

                # = = = = = = = = = = = = = = =
                
                list_note_2 = list_part[index_part].xpath('./descendant::span[contains(@title, "Please make sure")]/span/text()')
                note_2 = '' if len(list_note_2) == 0 else ';'.join([note_2.strip() for note_2 in list_note_2])

                # = = = = = = = = = = = = = = =

                src = list_part[index_part].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                src = '' if len(src) == 0 else 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')

                # = = = = = = = = = = = = = = =
                
                oe = list_part[index_part].xpath('./descendant::span[contains(@title, "Replaces these Alternate")]')
                oe = '' if len(oe) == 0 else oe[0].xpath('./text()')[0].strip().replace(', ', ';')
                oe = ';'.join(sorted(oe.split(';')))
                
                # = = = = = = = = = = = = = = =
                
                part_json = json.loads(list_part[index_part].xpath('./descendant::input[contains(@id, "listing_data_essential")]/@value')[0])
                part_key = part_json['partkey'].strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                                         'Kit_序号': int(input_.loc[a, 'Kit_No.']),
                                         'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                                         'Kit_Note': input_.loc[a, 'Kit_Note'],
                                         'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                         'Kit_Category': category,
                                         'Kit_Type': type_,
                                         'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                         'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number'],
                                         'Kit_Vehcile': '',
                                         'Kit_OE': oe,
                                         'Kit_Note_1': note_1,
                                         'Kit_Note_2': note_2,
                                         'Kit_Pic': '',
                                         'Kit_Url': url,
                                         'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + part_key,
                                         'Kit_Src': src,
                                         'Kit_Part_Key': part_key,
                                         'Kit_JOIN_TCPK': input_.loc[a, 'Kit_Type_Code']+';'+part_key}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                                     'Kit_No.': int(input_.loc[a, 'Kit_No.']),
                                     'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                                     'Kit_Note': input_.loc[a, 'Kit_Note'],
                                     'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                     'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                     'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'null':
            df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                                     'Kit_序号': int(input_.loc[a, 'Kit_No.']),
                                     'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                                     'Kit_Note': input_.loc[a, 'Kit_Note'],
                                     'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                     'Kit_Category': '',
                                     'Kit_Type': '',
                                     'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                     'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number'],
                                     'Kit_Vehcile': '',
                                     'Kit_OE': '',
                                     'Kit_Note_1': '',
                                     'Kit_Note_2': '',
                                     'Kit_Pic': '',
                                     'Kit_Url': '',
                                     'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_Src': '',
                                     'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_JOIN_TCPK': input_.loc[a, 'Kit_Type_Code']+';'+input_.loc[a, 'Kit_Part_Key']}])

            output_null = pd.concat([output_null, df_temp], ignore_index=True).fillna('')
    
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Info'] + ' - ' + input_.loc[a, 'Kit_No.'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./kit_part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info', 'Kit_No.'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./kit_part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
if len(output_null) != 0:
    output_null = output_null.drop_duplicates(ignore_index=True)
    output_null = output_null.sort_values(by=['Info', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
    output_null.to_excel('./kit_part_null-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
    print('爬虫存在null')
    print()
print('Done ~')

总数量：2892

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 2
[尝试次数：3] - [剩余数量：2885] - [当前时间：16:38:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 7
[尝试次数：3] - [剩余数量：2884] - [当前时间：16:38:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 8
[尝试次数：2] - [剩余数量：2883] - [当前时间：16:38:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 5
[尝试次数：9] - [剩余数量：2882] - [当前时间：16:38:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 4
[尝试次数：11] - [剩余数量：2881] - [当前时间：16:38:28]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110004 - 2
[尝试次数：5] - [剩余数量：2880] - [当前时间：16:38:29]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110004 - 1
[尝试次数：5] - [剩余数量：2879] - [当前时间：16:38:29]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 6
[尝试次数：16] - [剩余数量：2878] - [当前时间：16:38:31]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000 - 3
[尝试次

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454520 - 2
[尝试次数：4] - [剩余数量：2812] - [当前时间：16:39:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454516 - 6
[尝试次数：3] - [剩余数量：2811] - [当前时间：16:39:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454520 - 3
[尝试次数：2] - [剩余数量：2810] - [当前时间：16:39:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454520 - 6
[尝试次数：1] - [剩余数量：2809] - [当前时间：16:39:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454520 - 5
[尝试次数：3] - [剩余数量：2808] - [当前时间：16:39:11]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454524 - 3
[尝试次数：2] - [剩余数量：2807] - [当前时间：16:39:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454524 - 1
[尝试次数：4] - [剩余数量：2806] - [当前时间：16:39:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454524 - 4
[尝试次数：4] - [剩余数量：2805] - [当前时间：16:39:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454524 - 5
[尝试次数：1] - [剩余数量

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890264 - 7
[尝试次数：3] - [剩余数量：2740] - [当前时间：16:39:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890264 - 5
[尝试次数：5] - [剩余数量：2739] - [当前时间：16:39:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890268 - 1
[尝试次数：3] - [剩余数量：2738] - [当前时间：16:39:46]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890268 - 3
[尝试次数：1] - [剩余数量：2737] - [当前时间：16:39:47]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890268 - 5
[尝试次数：1] - [剩余数量：2736] - [当前时间：16:39:47]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890268 - 4
[尝试次数：1] - [剩余数量：2735] - [当前时间：16:39:48]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890268 - 6
[尝试次数：1] - [剩余数量：2734] - [当前时间：16:39:48]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454536 - 3
[尝试次数：2] - [剩余数量：2733] - [当前时间：16:39:48]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890268 - 2
[尝试次数：5] - [剩余数量

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 2
[尝试次数：1] - [剩余数量：2666] - [当前时间：16:40:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 6
[尝试次数：2] - [剩余数量：2665] - [当前时间：16:40:24]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 3
[尝试次数：5] - [剩余数量：2664] - [当前时间：16:40:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 1
[尝试次数：6] - [剩余数量：2663] - [当前时间：16:40:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 7
[尝试次数：1] - [剩余数量：2662] - [当前时间：16:40:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 5
[尝试次数：6] - [剩余数量：2661] - [当前时间：16:40:25]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890300 - 3
[尝试次数：1] - [剩余数量：2660] - [当前时间：16:40:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 4
[尝试次数：2] - [剩余数量：2659] - [当前时间：16:40:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304 - 8
[尝试次数：1] - [剩余数量

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158045 - 5
[尝试次数：4] - [剩余数量：2594] - [当前时间：16:40:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158045 - 2
[尝试次数：1] - [剩余数量：2593] - [当前时间：16:40:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158045 - 6
[尝试次数：2] - [剩余数量：2592] - [当前时间：16:40:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10981388 - 6
[尝试次数：1] - [剩余数量：2591] - [当前时间：16:41:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158045 - 7
[尝试次数：2] - [剩余数量：2590] - [当前时间：16:41:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158069 - 3
[尝试次数：1] - [剩余数量：2589] - [当前时间：16:41:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158085 - 2
[尝试次数：1] - [剩余数量：2588] - [当前时间：16:41:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158085 - 1
[尝试次数：4] - [剩余数量：2587] - [当前时间：16:41:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158085 - 4
[尝试次数：1] - [剩余数量

...

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305824 - 2
[尝试次数：1] - [剩余数量：338] - [当前时间：17:03:37]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305820 - 12
[尝试次数：1] - [剩余数量：337] - [当前时间：17:03:37]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305820 - 10
[尝试次数：1] - [剩余数量：336] - [当前时间：17:03:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305824 - 1
[尝试次数：1] - [剩余数量：335] - [当前时间：17:03:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305820 - 3
[尝试次数：6] - [剩余数量：334] - [当前时间：17:03:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305820 - 7
[尝试次数：12] - [剩余数量：333] - [当前时间：17:03:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305824 - 4
[尝试次数：2] - [剩余数量：332] - [当前时间：17:03:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305824 - 9
[尝试次数：1] - [剩余数量：331] - [当前时间：17:03:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305824 - 6
[尝试次数：2] - [剩余数量：330] - [当前时间：

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306036 - 1
[尝试次数：4] - [剩余数量：265] - [当前时间：17:04:24]

[null] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306036 - 4
[尝试次数：2] - [剩余数量：264] - [当前时间：17:04:25]

[null] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306036 - 3
[尝试次数：8] - [剩余数量：263] - [当前时间：17:04:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305956 - 2
[尝试次数：27] - [剩余数量：262] - [当前时间：17:04:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305956 - 6
[尝试次数：20] - [剩余数量：261] - [当前时间：17:04:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306036 - 5
[尝试次数：5] - [剩余数量：260] - [当前时间：17:04:26]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306156 - 1
[尝试次数：1] - [剩余数量：259] - [当前时间：17:04:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306156 - 3
[尝试次数：1] - [剩余数量：258] - [当前时间：17:04:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8306156 - 5
[尝试次数：1] - [剩余数量：257] - [当前

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921200 - 8
[尝试次数：6] - [剩余数量：191] - [当前时间：17:05:11]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8527960 - 10
[尝试次数：11] - [剩余数量：190] - [当前时间：17:05:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921204 - 4
[尝试次数：4] - [剩余数量：189] - [当前时间：17:05:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921200 - 4
[尝试次数：15] - [剩余数量：188] - [当前时间：17:05:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921204 - 3
[尝试次数：3] - [剩余数量：187] - [当前时间：17:05:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921204 - 8
[尝试次数：1] - [剩余数量：186] - [当前时间：17:05:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921204 - 7
[尝试次数：5] - [剩余数量：185] - [当前时间：17:05:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921208 - 1
[尝试次数：1] - [剩余数量：184] - [当前时间：17:05:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8921208 - 2
[尝试次数：4] - [剩余数量：183] - [当前时间：

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658988 - 11
[尝试次数：1] - [剩余数量：117] - [当前时间：17:05:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658988 - 9
[尝试次数：7] - [剩余数量：116] - [当前时间：17:05:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658988 - 10
[尝试次数：5] - [剩余数量：115] - [当前时间：17:05:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658992 - 2
[尝试次数：3] - [剩余数量：114] - [当前时间：17:06:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658992 - 1
[尝试次数：7] - [剩余数量：113] - [当前时间：17:06:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658992 - 3
[尝试次数：2] - [剩余数量：112] - [当前时间：17:06:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658992 - 4
[尝试次数：4] - [剩余数量：111] - [当前时间：17:06:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658992 - 6
[尝试次数：4] - [剩余数量：110] - [当前时间：17:06:03]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658996 - 1
[尝试次数：1] - [剩余数量：109] - [当前时间：1

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722080 - 4
[尝试次数：7] - [剩余数量：43] - [当前时间：17:06:35]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722084 - 3
[尝试次数：2] - [剩余数量：42] - [当前时间：17:06:36]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722076 - 7
[尝试次数：14] - [剩余数量：41] - [当前时间：17:06:37]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722088 - 2
[尝试次数：3] - [剩余数量：40] - [当前时间：17:06:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722088 - 3
[尝试次数：3] - [剩余数量：39] - [当前时间：17:06:38]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722084 - 2
[尝试次数：3] - [剩余数量：38] - [当前时间：17:06:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722084 - 4
[尝试次数：8] - [剩余数量：37] - [当前时间：17:06:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722088 - 4
[尝试次数：4] - [剩余数量：36] - [当前时间：17:06:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9854468 - 2
[尝试次数：1] - [剩余数量：35] - [当前时间：17:06:39]

